In [38]:
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="db-postgresql-sustainability-group-do-user-14262231-0.b.db.ondigitalocean.com",
    port=25060,
    database="defaultdb",
    user="doadmin",
    password="AVNS_xJlGYQTQBtMI5V_xcj1",
    sslmode="require"
)



In [31]:
industry_var = 'Financial Institution'
financial_institution_var = 'FALSE'
company_size_var = '>500 employees'
total_asset_var = ''
net_revenue_var = ''
listed_on_regulated_markets_var = 'Yes (Additional criteria for companies that do not fall into the mentioned industries)'
start_of_reporting_var = '2025 (for 2024)'
reporting_group_var = ''
hq_location_var = 'EU'

In [43]:
query

"SELECT d.directive_name FROM company as c JOIN directives d ON c.directive_id = d.directive_id WHERE c.industry LIKE '%Financial Institution%' AND c.financial_institution =  FALSE AND c.financial_institution LIKE '%FALSE%'"

In [85]:
import pandas as pd 

In [130]:
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="db-postgresql-sustainability-group-do-user-14262231-0.b.db.ondigitalocean.com",
    port=25060,
    database="defaultdb",
    user="doadmin",
    password="AVNS_xJlGYQTQBtMI5V_xcj1",
    sslmode="require"
)

list_industries = ['Financial Institution', 'Insurance', 'Pension Funds', 'Auditor', 'Extractive', 'Industry', 'Large Infrastructure', 'Company']
list_industries_sfdr = ['Investment firms', 'pension funds', 'asset managers', 'insurance companies', 'banks', 'venture capital funds', 'credit institutions offering portfolio management', 'financial advisors']
df_directives = pd.DataFrame(columns=['directive', 'start_date'])
start_of_rep = ''

cur = conn.cursor()

# query the csrd description 
query_csrd_name = "SELECT csrd_name FROM csrd"
query_csrd_description = "SELECT csrd_description FROM csrd"




# check for csrd
if company_size_var == '>500 employees' and (industry_var in list_industries or listed_on_regulated_markets_var == 'Yes'):
    df_directives.loc[len(df_directives)] = ['CSRD', '2025 (for 2024)']

elif (company_size_var == '>250 employees' and total_asset_var == '>20Mio') or (company_size_var == '>250 employees' and net_revenue_var == '>40Mio') or (total_asset_var == '>20Mio' and net_revenue_var == '>40Mio'): 
    df_directives.loc[len(df_directives)] = ['CSRD', '2026 (for 2025)']
    
elif (company_size_var == '<250 employees' and total_asset_var == '<20Mio' and listed_on_regulated_markets_var == 'Yes'):
    df_directives.loc[len(df_directives)] = ['CSRD', '2026 (for 2025)']

else:
    pass


# nfdr
if company_size_var == '>500 employees' and (industry_var in list_industries or listed_on_regulated_markets_var == 'Yes'):
    df_directives.loc[len(df_directives)] = ['NFRD', '2017 until 2025']


# EU-Taxonomy 
if company_size_var == '>500 employees' and (industry_var in list_industries or listed_on_regulated_markets_var == 'Yes'):
    df_directives.loc[len(df_directives)] = ['EU-Taxonomy', '2022']

# sfdr 
if industry_var in list_industries_sfdr:
    df_directives.loc[len(df_directives)] = ['SFDR Level 1', '2021']
    df_directives.loc[len(df_directives)] = ['SFDR Level 2', '2023']





result_df = pd.DataFrame(columns=['directive', 'start_date',  'csrd_name', 'csrd_description' ])


for index, row in df_directives.iterrows():

    directive = row['directive']
    start_date = row['start_date']


    if directive == 'CSRD':
        # Query the "csrd" table
        cur.execute("SELECT csrd_name FROM csrd")
        csrd_names = str(cur.fetchall())

        cur.execute("SELECT csrd_description FROM csrd")
        csrd_description = str(cur.fetchall())

        result_df.loc[len(result_df)] = ['CSRD', start_date, csrd_names, csrd_description]


    elif directive == 'NFRD':
        # Query the "nfrd" table
        cur.execute("SELECT nfrd_name FROM nfrd")
        nfrd_names = str(cur.fetchall())

        cur.execute("SELECT nfrd_description FROM nfrd")
        nfrd_description = str(cur.fetchall())

        result_df.loc[len(result_df)] = ['NFRD', start_date, nfrd_names, '']

    elif directive == 'EU-Taxonomy':
        # Query the "Eu-Taxonomy" table
        cur.execute("SELECT eu_taxonomy_name FROM eu_taxonomy")
        eu_taxonomy_names = str(cur.fetchall())

        cur.execute("SELECT eu_taxonomy_description FROM eu_taxonomy")
        eu_taxonomy_description = str(cur.fetchall())

        result_df.loc[len(result_df)] = ['EU-Taxonomy', start_date, eu_taxonomy_names, eu_taxonomy_description]

    elif directive == 'SFDR Level 1':
        # Query the "Eu-Taxonomy" table
        cur.execute("SELECT sfdr_name FROM sfdr WHERE sfdr_name LIKE 'Level 1%'")
        sfdr_names = str(cur.fetchall())

        cur.execute("SELECT sfdr_description FROM sfdr WHERE sfdr_name LIKE 'Level 1%'")
        sfdr_description = str(cur.fetchall())

        result_df.loc[len(result_df)] = ['SFDR Level 1', start_date, sfdr_names, '']

    elif directive == 'SFDR Level 2':
        # Query the "Eu-Taxonomy" table
        cur.execute("SELECT sfdr_name FROM sfdr WHERE sfdr_name LIKE 'Level 2%'")
        sfdr_names = str(cur.fetchall())

        cur.execute("SELECT sfdr_description FROM sfdr WHERE sfdr_name LIKE 'Level 2%'")
        sfdr_description = str(cur.fetchall())

        result_df.loc[len(result_df)] = ['SFDR Level 2', start_date, sfdr_names, sfdr_description]


    else: 
        break

# Close the cursor and connection
cur.close()
conn.close()




In [131]:
result_df

,directive,start_date,csrd_name,csrd_description
0,CSRD,2025 (for 2024),"[('General principles',), ('Double materiality...","[('General, strategy, governance and materiali..."
1,NFRD,2017 until 2025,"[('Environmental issues',), ('Social issues an...",
2,EU-Taxonomy,2022,[('Environmentally sustainable portion of the ...,"[('',), ('Broken down by taxonomy compliant ec..."
3,SFDR Level 1,2021,[('Level 1: List of sustainability / ESG risks...,
4,SFDR Level 2,2023,[('Level 2: Extent to which each product compl...,[('Article 6-products are those that only asse...


In [82]:
list_directives = [[]]
list_directives.append([['CSRD'], ['2025 (for 2024)']])


In [84]:
list_directives[0][1]

IndexError: list index out of range

In [ ]:

# Construct the query strings
query = "SELECT d.directive_name FROM company as c JOIN directives d ON c.directive_id = d.directive_id WHERE c.industry LIKE '%" + industry_var +"%' AND c.financial_institution =  " + financial_institution_var +" AND c.company_size LIKE '%" + company_size_var +"%' AND c.total_asset LIKE '%" + total_asset_var +"%' AND c.net_revenue LIKE '%" + net_revenue_var +"%' AND c.listed_on_regulated_markets LIKE '%" + listed_on_regulated_markets_var +"%' AND c.start_of_reporting LIKE '%" + start_of_reporting_var +"%' AND c.reporting_group LIKE '%" + reporting_group_var +"%' AND c.hq_location LIKE '%" + hq_location_var +"%'"

query_csrd = "SELECT csrd_name, csrd_description FROM csrd"

query_nfdr = "SELECT nfdr_name, nfdr_description FROM nfdr"
            
query_eutax = "SELECT eutax_name, eutax_description FROM eu_taxonomy"

query_sfdr = "SELECT sfdr_name, sfdr_description FROM sfdr"		


# Query the "Company" table
cur.execute(query)
dicrective_rows = cur.fetchone()

for row in dicrective_rows:
    directive = row

if directive == 'CSRD':
    cur.execute(query_csrd)
    result_rows = cur.fetchall()


# Close the cursor and connection
cur.close()
conn.close()

In [ ]:
#initiate three dimensional array à [CSRD:][Component Heading – line break – Component #description if available][Year to file] 
 
csrd_array= empty_array 
if (company_sector mentioned in industry column): 
Read line where company sector is mentioned, and store start of reporting + general components in csrd_array 
 
else if(at least 2 out of 3 criteria of company level id 2 are met): 
Read line of id 2 and store start of reporting + general components in csrd_array 
 
else if(at least 2 out of 3 criteria of company level id 3 are met): 
Read line of id 3 and store start of reporting + general components in csrd_array 
 
else: 
Read line of id 4 and store start of reporting + general components in csrd_array 

In [61]:
result_rows

[('General principles',
  'General, strategy, governance and materiality assessment disclosure requirements'),
 ('Double materiality',
  'How companies operations impact people and the environment, and how sustainability-related developments impact the organisation'),
 ('Environment',
  'Climate change, Pollution, Water and marine resources, Biodiversity and ecosystems und Resource use and circular economy'),
 ('Detailed energy consumption data', ''),
 ('Scope 1 GHG emissions (actual and forecast)',
  'company facilities, company vehicles'),
 ('Scope 2 GHG emissions (actual and forecast)',
  'purchased electricity, steam, heating  & cooling for own use'),
 ('Scope 3 upstream GHG emissions (actual and forecast)',
  'purchased goods and services, capital goods, fuel and energy related activities'),
 ('Scope 3 downstream GHG emissions (actual and forecast)',
  'transportation and distribution, processing of sold products, end of life treatment of sold products, leased assets, franchises')

In [ ]:
# Query the "csrd" table
cur.execute("SELECT * FROM csrd")
csrd_rows = cur.fetchall()
print("csrd:")
for row in csrd_rows:
    print(row)

In [ ]:
 AND c.financial_institution = ''' || financial_institution_var || '''                   
                   AND c.company_size LIKE ''%' || company_size_var || '%''
                   AND c.total_asset LIKE ''%' || total_asset_var || '%''				   
                   AND c.net_revenue LIKE ''%' || net_revenue_var || '%''				   
                   AND c.listed_on_regulated_markets LIKE ''%' || listed_on_regulated_markets_var || '%''				   			   				   
				   AND c.start_of_reporting LIKE ''%' || start_of_reporting_var || '%''				   
				   AND c.reporting_group LIKE ''%' || reporting_group_var || '%''				   
                   AND c.hq_location LIKE ''%' || hq_location_var || '%''